In [1]:
import numpy as np
import csv
import pandas as pd
import pygad
from src import  repeat_elements, tot_TCI_multiple_reactors,  generate_random_list, list_of_potential_number_of_reactors
import matplotlib.pyplot as plt
from scheduleOptimizationForReactorMix import optimize_schedule, capacity_factor_weeks_approach_mix_reactors
import warnings
import time
warnings.filterwarnings('ignore')
import random


5


In [2]:
# TCI for a mix of reactors

def tot_TCI_multiple_reactors_mix (power_list, interest_rate, num_reactors_list):
    tot_TCI_for_specific_reactor_power_list = []
    
    for i in range(len(power_list)):
        tot_TCI_for_specific_reactor_power = tot_TCI_multiple_reactors (power_list[i], interest_rate, num_reactors_list[i])
        tot_TCI_for_specific_reactor_power_list.append(tot_TCI_for_specific_reactor_power)
        
    return sum(tot_TCI_for_specific_reactor_power_list)

In [3]:
def level_cost_of_energy_reactor_mix( interest_rate, power_list, num_reactors_list,\
    list_of_generated_MWh_per_year_from_all_reactors_per_demand, list_of_sold_electricity_MWh_per_year_from_all_reactors, elec_price,\
        list_of_OM_cost_per_year_all_reactors):
    
    sum_cost = 0 # initialization 
    sum_elec = 0 # initialization 
           
    for year in range( len(list_of_generated_MWh_per_year_from_all_reactors_per_demand)):
        if year == 0:
            cap_cost =  tot_TCI_multiple_reactors_mix(power_list, interest_rate, num_reactors_list)
            OM_cost_per_year = 0
            elec_gen = 0
            revenue = 0
        
        elif year > 0:
         
            cap_cost = 0 
            
            OM_cost_per_year =  list_of_OM_cost_per_year_all_reactors[year-1]
            revenue = elec_price * list_of_sold_electricity_MWh_per_year_from_all_reactors[year-1]
            elec_gen =  list_of_generated_MWh_per_year_from_all_reactors_per_demand[year-1]
        
        sum_cost += (cap_cost + OM_cost_per_year - revenue) / ((1 +interest_rate)**(year) ) 
        sum_elec += elec_gen/ ((1 + interest_rate)**year) 
    
    LCOE =  sum_cost/ sum_elec
    # print("LCOE NOW is: ", LCOE)
    return LCOE

# Initialize the solution

In [5]:
def initial_population_reactors(power_list, demand, sol_per_pop):
    
    rough_multiplier =  int(np.ceil(demand / sum(power_list)))
    initial_list_of_multipliers = [ rough_multiplier-1,  rough_multiplier, rough_multiplier+1]
    
    # Define the length of each list you want to create
    list_length = len(power_list)
    
    
    # Number of lists to create
    num_lists = sol_per_pop
    
    # Store the lists in a container (e.g., a list of lists)
    lists_of_lists = []

    # Create lists
    for _ in range(num_lists):
        # list1 = generate_random_list(min(initial_list_of_multipliers ), max(initial_list_of_multipliers ), list_length)
        list1= [0]*list_length
        lists_of_lists.append(list1)
       
#     # # specify the gene space by limiting the possible number of solitions
#     number_of_reactors_list_of_lists = []
    
#     for i in range(len(power_list )):
#         number_of_reactors_list = list_of_potential_number_of_reactors(power_list[i], demand)
#         number_of_reactors_list_of_lists.append(number_of_reactors_list)
#     # upper_limit_list = []
#     # for i in range(len(power_list )):
#     #     if power_list[i] > 500:
#     #         # I use the multiplier 1.05 because of the ratio between refueling duration and operational lifetime is 0.05
#     #         upper_limit =range(int(np.ceil(1.05*demand/power_list[i])))
#     #     elif power_list[i] <= 500 and power_list[i]>50:  
#     #         upper_limit =range(int(np.ceil(1.03*demand/power_list[i])))
#     #     elif power_list[i] <= 500 and power_list[i]<50:    
#     #          upper_limit =range(int(np.ceil(1.02*demand/power_list[i]))) 
#     #     upper_limit_list.append(upper_limit)    
        
    return lists_of_lists 







def on_gen(ga_instance):
    # pass
    print("Generation : ", ga_instance.generations_completed,  ga_instance.best_solution()[0], ga_instance.best_solution()[1])

def optimize_lcoe(power_list,  levelization_period_weeks, demand , interest_rate, capacity_factor_t_min_criteria):
   
    start_time = time.time()
    
    # GA params
    sol_per_pop = 2*int(np.ceil((len(power_list))) ) 
    
  
    
    
    

    num_generations = 6000
    num_parents_mating =  4 # consider increasing this
    num_genes = len(power_list)
    


    parent_selection_type = "rank"
    keep_parents =  1

    # crossover_type = "single_point"

    mutation_type = "adaptive"
    mutation_percent_genes =  mutation_percent_genes = (20, 1) #10
    
    
    
    
    
    
     
    initial_pop  = initial_population_reactors(power_list, demand, sol_per_pop)
    # potential_solutions = initial_population_reactors(power_list, demand, sol_per_pop)[1]
  
    
    # potential solutions (Gene Space)
    solutions_P_list = []
    for i in range(len(power_list)):
        solutions_P = list_of_potential_number_of_reactors(power_list[i], demand)
        solutions_P_list.append(solutions_P)
    
    allow_dup  = True

    def fitness_eq(output_discrepancy):
        return -100 / (output_discrepancy) 
        
    
    def fitness_func(ga_instance, solution, solution_idx):
       
        power_list_modified = [power_list [i] for i in range(len(solution)) if solution[i] != 0]
      
        Num_of_each_reactor_type_modified = [x for x in solution if x != 0]
      
        long_list_power = repeat_elements(power_list_modified, Num_of_each_reactor_type_modified)
       
        
        if sum(long_list_power) >= demand * capacity_factor_t_min_criteria  and sum(long_list_power) <=2*demand: #  ((sum(long_list_power)) - min(long_list_power) ) <= demand:
            
            
        
                capacity_factor_results =   capacity_factor_weeks_approach_mix_reactors(long_list_power  ,levelization_period_weeks, demand)
                MWh_generated_per_year_per_demand_list = capacity_factor_results[4]
                MWh_excess_per_year_list =     capacity_factor_results[5]
                Tot_OM__cost_per_year_list =  capacity_factor_results[6]
                
                capacity_factor_t_min = min(capacity_factor_results[1])
               
                output_lcoe = level_cost_of_energy_reactor_mix( interest_rate, power_list_modified, Num_of_each_reactor_type_modified,\
            MWh_generated_per_year_per_demand_list, MWh_excess_per_year_list, 0,\
                Tot_OM__cost_per_year_list)

                
                # if min_tot_P(power_list , solution, levelization_period_weeks) == expected_out :
                # fitness = 1 / ( np.abs(output - expected_out)  +1)
            
                fitness = fitness_eq(0 -  output_lcoe) # Here I assume the target is a very small number (zero$/MWh)
            
                if capacity_factor_t_min < capacity_factor_t_min_criteria: # Must satsify the criteria
                    fitness = -10000
                
         
         
                
                print("solution is : " , solution)
                print("Total Power is : " , sum(long_list_power))
                print("MW discrepancy : " , np.abs(demand - sum(long_list_power)))
                print("Capacity Factor (min): " , np.round(capacity_factor_t_min, 3))
                print("LCOE : " , np.round(output_lcoe, 3))
            
         
        
        else:
            fitness = -100000
        if fitness >=0:    
            print("FITNESS NOW: ", fitness, "\n")
        return fitness
            
    ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating= num_parents_mating,
                       fitness_func= fitness_func,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                     
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents, # crossover_type=crossover_type,
                       
                       mutation_percent_genes= mutation_percent_genes,
                       gene_space= solutions_P_list,
                       mutation_type=mutation_type, # stop_criteria= ["saturate_20"], # 
                      
                       on_generation= on_gen,
                        fitness_batch_size=1,
                        keep_elitism = int(sol_per_pop/5), # crossover_probability = 0.7,
                        gene_type = int, 
                        initial_population = initial_pop,
                        parallel_processing=8,
                    
                    
                    
                       allow_duplicate_genes=allow_dup )
      
    
 
    ga_instance.run()
  
    end_time = time.time() 
    
    
    sol, sol_fitness, _ = ga_instance.best_solution()
    print("\n The optimization program runtime is " , np.round( (end_time -start_time), 0), " sec", " & The Number of Generations Passed is ",\
        ga_instance.generations_completed, "...... Fitness value of the best solution = {solution_fitness}".format(solution_fitness=sol_fitness)) 


    
    return sol







In [6]:
optimize_lcoe( [1000, 300,  50] ,  int(30*365/7), 3000 , 0.06, 0.99)
    


 The program runtime is  1.0  sec  & The Number of Generations Passed is  1 ...... Fitness value of the best solution = 1.0

 The difference between prediction and desired output is 0 : 2000 vs. 2000 Total MW

It takes 0.15 years to startup up all the reactors 

solution is :  [3 0 0]
Total Power is :  3000
MW discrepancy :  0
Capacity Factor (min):  0.667
LCOE :  114.701

 The program runtime is  1.0  sec  & The Number of Generations Passed is  1 ...... Fitness value of the best solution = 1.0

 The program runtime is  1.0  sec  & The Number of Generations Passed is  1 ...... Fitness value of the best solution = 1.0

 The difference between prediction and desired output is 0 : 3000 vs. 3000 Total MW

It takes 0.23 years to startup up all the reactors 

solution is :  [4 0 0]
Total Power is :  4000
MW discrepancy :  1000
Capacity Factor (min):  1.0
LCOE :  145.746
FITNESS NOW:  0.6861264414003431 


 The difference between prediction and desired output is 0 : 3000 vs. 3000 Total MW

I